In [1]:
print("hello")

hello


In [40]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [4]:
%pwd

'c:\\Users\\USER\\Projects\\SourceCodeAnalysis\\research'

In [3]:
!mkdir test_repo

In [ ]:
repo_path = "text_repo/"
repo = Repo.clone_from("https://github.com/Priya-Rathor/InterviewQuesCreatorAPP",to_path=repo_path)

In [43]:
loader = GenericLoader.from_filesystem(repo_path, glob="**/*",
                                       suffixes=[".py"],
                                       parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
                                       )

In [7]:
documents = loader.load()

In [8]:
documents

[Document(page_content='from fastapi import FastAPI, Form, Request, Response, File, Depends, HTTPException, status\nfrom fastapi.responses import RedirectResponse\nfrom fastapi.staticfiles import StaticFiles\nfrom fastapi.templating import Jinja2Templates\nfrom fastapi.encoders import jsonable_encoder\nimport uvicorn\nimport os\nimport aiofiles\nimport json\nimport csv\nfrom src.helper import llm_pipeline\n\n\napp = FastAPI()\napp.mount("/static", StaticFiles(directory="static"), name="static")\n\ntemplates = Jinja2Templates(directory="templates")\n\n\n@app.get("/")\nasync def index(request: Request):\n    return templates.TemplateResponse("index.html", {"request": request})\n\n\n@app.post("/upload")\nasync def chat(request: Request, pdf_file: bytes = File(), filename: str = Form(...)):\n    base_folder = \'static/docs/\'\n    if not os.path.isdir(base_folder):\n        os.mkdir(base_folder)\n    pdf_filename = os.path.join(base_folder, filename)\n\n    async with aiofiles.open(pdf_fil

In [9]:
len(documents)

5

In [10]:
documents[2]

Document(page_content='from langchain_community.document_loaders import PyPDFLoader\nfrom langchain_community.embeddings import OpenAIEmbeddings\nfrom langchain_community.vectorstores import FAISS\nfrom langchain_community.chat_models import ChatOpenAI\nfrom langchain.prompts import PromptTemplate\nfrom langchain.chains.summarize import load_summarize_chain\nfrom langchain.embeddings.openai import OpenAIEmbeddings\nfrom langchain.vectorstores import FAISS\nfrom langchain.chains import RetrievalQA\nimport os\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nfrom langchain.text_splitter import TokenTextSplitter\nfrom langchain.schema import Document\n\n\n# OpenAI authentication\nload_dotenv()\nOPENAI_API_KEY = os.getenv("OPENAI_API_KEY")\nos.environ["OPENAI_API_KEY"] = OPENAI_API_KEY\n\n\n\ndef file_processing(file_path):\n\n    # Load data from PDF\n    loader = PyPDFLoader(file_path)\n    data = loader.load()\n\n    question_gen = \'\'\n\n    for page in data:\n        questio

In [44]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language= Language.PYTHON,
                                             chunk_size =500,
                                             chunk_overlap=20)

In [12]:
documents_splitter

In [45]:
texts = documents_splitter.split_documents(documents)

In [14]:
texts

[Document(page_content='from fastapi import FastAPI, Form, Request, Response, File, Depends, HTTPException, status\nfrom fastapi.responses import RedirectResponse\nfrom fastapi.staticfiles import StaticFiles\nfrom fastapi.templating import Jinja2Templates\nfrom fastapi.encoders import jsonable_encoder\nimport uvicorn\nimport os\nimport aiofiles\nimport json\nimport csv\nfrom src.helper import llm_pipeline\n\n\napp = FastAPI()\napp.mount("/static", StaticFiles(directory="static"), name="static")', metadata={'source': 'text_repo\\app.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='templates = Jinja2Templates(directory="templates")\n\n\n@app.get("/")\nasync def index(request: Request):\n    return templates.TemplateResponse("index.html", {"request": request})\n\n\n@app.post("/upload")\nasync def chat(request: Request, pdf_file: bytes = File(), filename: str = Form(...)):\n    base_folder = \'static/docs/\'\n    if not os.path.isdir(base_folder):\n        os.mkdir(b

In [15]:
len(texts)

19

In [46]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [47]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [48]:
embedding = OpenAIEmbeddings(disallowed_special=())

In [49]:
vectorstore = FAISS.from_documents(texts,embedding)

In [59]:
vectordb = FAISS.from_documents(texts, embedding=embedding)
vectordb.save_local("./faiss_index")

In [55]:
llm = ChatOpenAI()

In [60]:
memory = ConversationSummaryMemory(llm = llm, memory_key="chat_history",return_messages=True)

In [61]:
qa = ConversationalRetrievalChain.from_llm(llm,retriever=vectordb.as_retriever(searxh_type="mmr",search_kwargs={"k":8}),memory=memory)

In [62]:
question = "What is Uvicon function? "

In [63]:
result = qa(question)
print(result['answer'])

Uvicorn is a lightning-fast ASGI server implementation, using uvloop and httptools. It runs asynchronous Python web code in a single process.


In [64]:
question = "What is app.py file do?"
result = qa(question)
print(result['answer'])

The `app.py` file defines a FastAPI application that serves two endpoints: `/analyze` and `/upload`. 

The `/analyze` endpoint is a POST request that takes a `pdf_filename` parameter from a form and calls the `get_csv` function to get an output file. The response includes the output file in JSON format.

The `/upload` endpoint is a POST request that takes a PDF file and a filename parameter from a form. It saves the uploaded PDF file to a specific folder in the project directory.

Additionally, the file sets up a FastAPI application with endpoints for serving static files and rendering HTML templates using Jinja2Templates. The application is then run using uvicorn with the specified host and port.
